In [1]:
pip install flask


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, render_template, request, session
from werkzeug.utils import secure_filename
import os
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

app = Flask(__name__)
app.secret_key = 'your_secret_key'

UPLOAD_FOLDER = 'csv_files'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload directory exists
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

def generate_pie_chart(labels, values):
    trace = go.Pie(labels=labels, values=values)
    data = [trace]
    layout = go.Layout(title='Pie Chart')
    fig = go.Figure(data=data, layout=layout)
    return fig.to_html(full_html=False)

def generate_scatter_plot(x_data, y_data, x_label, y_label):
    trace = go.Scatter(x=x_data, y=y_data, mode='markers')
    layout = go.Layout(title='Scatter Plot', xaxis=dict(title=x_label), yaxis=dict(title=y_label))
    fig = go.Figure(data=[trace], layout=layout)
    return fig.to_html(full_html=False)

def generate_bar_chart(df, categorical_column):
    fig = px.bar(df, x=categorical_column, title=f'Bar Chart - {categorical_column}',
                 labels={categorical_column: categorical_column},
                 color_discrete_sequence=['#1f77b4'])  # Set color of bars
    
    fig.update_traces(marker_line_color='rgb(8,48,107)',  # Set marker line color
                      marker_line_width=1.5,  # Set marker line width
                      opacity=0.8)  # Set opacity of bars

    fig.update_layout(
        xaxis_title=categorical_column,
        yaxis_title='Count',
        font=dict(size=14, color='black'),  # Set font size and color
        title_font=dict(size=18, color='black'),  # Set title font size and color
        plot_bgcolor='white',  # Set plot background color
        showlegend=False,  # Hide legend
        margin=dict(l=40, r=40, t=50, b=40)  # Set margins
    )

    return fig.to_html(full_html=False)



def generate_histogram(df, numerical_column):
    fig = px.histogram(df, x=numerical_column, title=f'Histogram - {numerical_column}',
                       labels={numerical_column: numerical_column},
                       color_discrete_sequence=['#1f77b4'])  # Set color of bars
    
    fig.update_traces(marker_line_color='rgb(8,48,107)',  # Set marker line color
                      marker_line_width=1.5,  # Set marker line width
                      opacity=0.8)  # Set opacity of bars

    fig.update_layout(
        xaxis_title=numerical_column,
        yaxis_title='Frequency',
        font=dict(size=14, color='black'),  # Set font size and color
        title_font=dict(size=18, color='black'),  # Set title font size and color
        plot_bgcolor='white',  # Set plot background color
        showlegend=False,  # Hide legend
        margin=dict(l=40, r=40, t=50, b=40)  # Set margins
    )

    return fig.to_html(full_html=False)




@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return render_template('upload.html', error='No file part')

        file = request.files['file']

        if file.filename == '':
            return render_template('upload.html', error='No selected file')

        if file and file.filename.endswith('.csv'):
            # Save the uploaded file
            filename = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename))
            file.save(filename)
            session['csv_file'] = filename

            # Read the data from the uploaded CSV file
            df = pd.read_csv(filename)
            # Convert DataFrame to HTML table
            data_html = df.to_html(classes='table table-striped', index=False)

            return render_template('upload.html', success='File uploaded successfully.', data_html=data_html)
        else:
            return render_template('upload.html', error='Invalid file type. Please upload a CSV file.')

    return render_template('upload.html')

@app.route('/visualization', methods=['GET', 'POST'])
def visualize_data():
    # Check if the CSV file path is stored in the session
    if 'csv_file' not in session:
        return render_template('visualization.html', error='No CSV file uploaded. Please upload a CSV file first.')

    # Read DataFrame from the uploaded CSV file
    df = pd.read_csv(session['csv_file'])
    columns = df.columns.tolist()

    if request.method == 'POST':
        vis_type = request.form['type']

        if vis_type == 'pie':
            category_column = request.form['category_column']
            value_column = request.form['value_column']
            labels = df[category_column]
            values = df[value_column]
            chart = generate_pie_chart(labels, values)
            return render_template('visualization.html', chart=chart, columns=columns)

        elif vis_type == 'scatter':
            x_column = request.form['x_column']
            y_column = request.form['y_column']
            x_data = df[x_column]
            y_data = df[y_column]
            x_label = x_column
            y_label = y_column
            chart = generate_scatter_plot(x_data, y_data, x_label, y_label)
            return render_template('visualization.html', chart=chart, columns=columns)

        elif vis_type == 'bar':
            categorical_column = request.form['categorical_column']
            chart = generate_bar_chart(df, categorical_column)
            return render_template('visualization.html', chart=chart, columns=columns)
        
        elif vis_type == 'histogram':
            numerical_column = request.form['numerical_column']
            chart = generate_histogram(df, numerical_column)
            return render_template('visualization.html', chart=chart, columns=columns)


    return render_template('visualization.html', columns=columns)



def custom_pivot(df, index_column, category_column, value_column, agg_function):
    pivot_table = pd.pivot_table(df, index=index_column, columns=category_column, values=value_column, aggfunc=agg_function)
    return pivot_table

@app.route('/data', methods=['GET'])
def pivot_table():
    # Directory containing CSV files
    csv_dir = 'csv_files'

    # List all CSV files in the directory
    csv_files = [file for file in os.listdir(csv_dir) if file.endswith('.csv')]

    # Use the first CSV file found in the directory
    if csv_files:
        csv_file_path = os.path.join(csv_dir, csv_files[0])
        df = pd.read_csv(csv_file_path)

        # Define pivot parameters
        index_column = 'index_column_name'
        category_column = 'category_column_name'
        value_column = 'value_column_name'
          # Choose aggregation function, e.g., 'sum', 'mean', 'count', etc.

        # Generate pivot table
        pivot_result = custom_pivot(df, index_column, category_column, value_column, )

        # Convert pivot table to HTML
        pivot_table_html = pivot_result.to_html(classes='table table-striped')
    else:
        pivot_table_html = '<p>No CSV files found in the directory.</p>'

    return render_template('data.html', pivot_table_html=pivot_table_html)




if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
